In [1]:
import pandas as pd
from crewai import Agent, Task, Crew
import sys
import os
from IPython.display import display, Markdown
import re
from datetime import datetime
import json
from langchain.chat_models import ChatOpenAI

In [2]:
#load data (so can reference as needed)
df = pd.read_csv("data/prostate_patient_data.csv")

In [3]:
#access api
os.environ["OPENROUTER_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
os.environ["OPENAI_API_KEY"] = os.environ["OPENROUTER_API_KEY"]

In [4]:
from crew_agents.summarizer_agent import summarizer_agent
from crew_agents.validator_agent import validator_agent
from crew_agents.planner_agent import planner_agent
from crew_agents.tools import EvidenceLookupTool
from crew_agents.tools import summarize_patient_context
from crew_agents.tools import TrajectoryEvidenceTool



c:\Users\zcali\Capstone\CrewAI_Draft_Model\crew_agents\summarizer_agent.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name = "gpt-4-turbo", temperature = 0)


In [5]:
from crew_agents.tools import summarize_patient_context
print(summarize_patient_context._run(patient_id=1))


[DEBUG] Context fetched from MCP API for patient 1:
2020-03-13: DIAGNOSIS - Initial PSA=14.7 ng/mL, PI-RADS=5, Initial Weight=73.7 kg
2020-03-13: Treatment=ADT, PI-RADS=5, PSA=14.7 ng/mL, Weight=73.7 kg, Bone Pain=Mild
2020-06-11: Treatment=ADT, PI-RADS=5, PSA=14.5 ng/mL, Weight=74.5 kg, Bone Pain=None
2020-09-09: Treatment=ADT, PI-RADS=5, PSA=14.4 ng/mL, Weight=75.3 kg, Bone Pain=Mild
2020-12-08: Treatment=ADT, PI-RADS=5, PSA=14.1 ng/mL, Weight=76.2 kg, Bone Pain=Mild
2021-03-08: Treatment=ADT, PI-RADS=5, PSA=13.9 ng/mL, Weight=76.9 kg, Bone Pain=Mild
2021-06-06: Treatment=ADT, PI-RADS=5, PSA=13.6 ng/mL, Weight=77.7 kg, Bone Pain=Mild
2020-03-13: DIAGNOSIS - Initial PSA=14.7 ng/mL, PI-RADS=5, Initial Weight=73.7 kg
2020-03-13: Treatment=ADT, PI-RADS=5, PSA=14.7 ng/mL, Weight=73.7 kg, Bone Pain=Mild
2020-06-11: Treatment=ADT, PI-RADS=5, PSA=14.5 ng/mL, Weight=74.5 kg, Bone Pain=None
2020-09-09: Treatment=ADT, PI-RADS=5, PSA=14.4 ng/mL, Weight=75.3 kg, Bone Pain=Mild
2020-12-08: Treatme

In [6]:
#trying to get hallucinations under control
def remove_hallucinated_demographics(text: str) -> str:
    filtered_lines = []
    for line in text.splitlines():
        if re.search(r'(John Doe|Date of Birth|Gender:|Phone:|Email:)', line):
            continue
        filtered_lines.append(line)
    return '\n'.join(filtered_lines)

In [7]:
#Fix EvidenceLookup errors
def append_evidencelookup_guidance(text):
    return text + (
        "\n\n To use the EvidenceLookup tool correctly, input must be a single quoted string, e.g., 'PIRADS 4 prostate cancer risk'.\n"
        "Do NOT call the tool with an empty object ({}).\n"
        "Avoid malformed calls like JSON objects; only plain English strings are accepted."
    )

In [8]:
#define tasks w changeable patient_id

#clinical summary pipeline

def run_pipeline(patient_id: str):

    mcp_context = summarize_patient_context._run(patient_id=patient_id)

    planner_task = Task(
    description=(
        f"Use ONLY the structured clinical entries below to construct a timeline of events for patient ID {patient_id}.\n\n"
        f"--- BEGIN STRUCTURED ENTRIES ---\n{mcp_context}\n--- END STRUCTURED ENTRIES ---\n\n"
        "**Rules:**\n"
        "- Do NOT invent events, symptoms, scans, or dates not included above.\n"
        "- Each timeline item must match a real dated entry.\n"
        "- Format each as: - [Date]: [Event Type] — [Details]."
    ),
    expected_output="A clean, strictly factual timeline of clinical events derived from the structured entries.",
    agent=planner_agent,
    input_data=mcp_context
)


    #also added a lot here to prevent hallucinations
    mcp_context = summarize_patient_context._run(patient_id=patient_id)
    rag_evidence = TrajectoryEvidenceTool()._run({"mcp_context": mcp_context})

    rag_paragraph = f"\n\n**2. Clinical Literature Context**\n{rag_evidence}\n"

    summarizer_task = Task(
    description = (
        f"You are writing a clinical report for patient ID {patient_id} based solely on the structured clinical context below.\n\n"
        f"--- BEGIN PATIENT CONTEXT ---\n{mcp_context}\n--- END PATIENT CONTEXT ---\n\n"

        "**Instructions:**\n"
        "- ONLY use the information found between the dashed lines.\n"
        "- DO NOT invent or guess any additional findings, labs, scans, treatments, or diagnoses.\n"
        "- You MUST call the TrajectoryEvidenceTool to retrieve one relevant clinical study.\n"
        "- Cite the study using the format: 'According to [Author, Year](URL), ... (from *Study Title*)'.\n"
        "- Write this paragraph professionally as if documenting for an oncologist.\n"
        "- Use EvidenceLookupTool only to clarify ambiguous terms (e.g., PIRADS).\n"
        "- Include any weight values present in both the summary and timeline.\n"
        "- If multiple weights appear, summarize the trend (e.g., weight gain from X kg to Y kg).\n\n"

        "**Your Output Must Include:**\n"
        "1. **Clinical Summary for Patient {patient_id}** — A 3–5 sentence summary of the structured clinical data.\n"
        "2. **Clinical Literature Context** — A paragraph citing relevant research using TrajectoryEvidenceTool.\n"
        "3. **Patient Timeline** — Chronological bullet points of dated clinical events.\n"
        "4. **Validation Comments** — Flag hallucinations, missing values, or inconsistencies.\n\n"
        "**Do NOT fabricate studies or citations. Only cite what the tool provides.**"
    ),
    expected_output = "Four structured sections: Clinical Summary, Clinical Literature Context, Patient Timeline, Validation Comments.",
    tools = [EvidenceLookupTool(), TrajectoryEvidenceTool()],
    agent = summarizer_agent
)

    summarizer_task.description = append_evidencelookup_guidance(summarizer_task.description)

    #Want to make sure it doesn't miss any provided values, additional hallucinatio checks
    validator_task = Task(
        description=(
            f"You are validating the summarizer output for patient ID {patient_id}.\n\n"
            "**1. Hallucination Check**\n"
            "- Flag any fabricated labs (e.g. CBC, lipid panel), scans (e.g. MRI brain, CT chest), meds (e.g. amoxicillin, lisinopril), or therapy.\n"
            "**2. Missing Data Check**\n"
            "- REQUIRED if present in MCP summary: PSA, PIRADS, ADT, Radiation, Surgery, Weight, Bone Pain, etc.\n"
            "**3. Final Format Check**\n"
            "- Must include: Clinical Summary for Patient {patient_id}, Patient Timeline, Validation Comments.\n\n"
            "**Final Output Format**\n"
            "**Hallucinated Content List**: ...\n"
            "**Missing Data List**: ...\n"
            "**Overall Validity Summary**: ..."
        ),
        expected_output = "Three sections: Hallucinated Content List, Missing Data List, Overall Validity Summary.",
        agent = validator_agent
    )

    #define crew and add loop to fix what validator flags
    max_attempts = 3
    for attempt in range(max_attempts):
        crew = Crew(
            agents=[planner_agent, summarizer_agent, validator_agent],
            tasks=[planner_task, summarizer_task, validator_task],
            verbose=True
        )

        
        results = crew.kickoff()

        summary_text = ""
        validator_output = ""
        for task_output in results.tasks_output:
            if task_output.agent == "LLM Clinical Summarizer" and "Clinical Summary for Patient" in task_output.raw:
                summary_text = task_output.raw
            elif task_output.agent == "Clinical Output Validator":
                validator_output = task_output.raw
        #have validator agent check for missing or fake data
        hallucinated = "Hallucinated Content List: None" not in validator_output
        missing_data = "Missing Data List: None" not in validator_output
        #if none, proceed!
        if not hallucinated and not missing_data:
            break
        #otherwise retry
        summarizer_task.description += (
            "\n\n Previous output was flagged for hallucinations or missing key data. Only include information from structured fields. Re-verify presence of PSA, PIRADS, weight, bone pain."
        )
    #make sure not adding "John Doe" or fake doctor name
    cleaned_results = remove_hallucinated_demographics(summary_text)
    #print
    display(Markdown(f"### Summary for Patient P{int(patient_id)}\n\n{cleaned_results}"))

In [9]:
#try for patient 1 - can input any number!
run_pipeline("1")

[DEBUG] Context fetched from MCP API for patient 1:
2020-03-13: DIAGNOSIS - Initial PSA=14.7 ng/mL, PI-RADS=5, Initial Weight=73.7 kg
2020-03-13: Treatment=ADT, PI-RADS=5, PSA=14.7 ng/mL, Weight=73.7 kg, Bone Pain=Mild
2020-06-11: Treatment=ADT, PI-RADS=5, PSA=14.5 ng/mL, Weight=74.5 kg, Bone Pain=None
2020-09-09: Treatment=ADT, PI-RADS=5, PSA=14.4 ng/mL, Weight=75.3 kg, Bone Pain=Mild
2020-12-08: Treatment=ADT, PI-RADS=5, PSA=14.1 ng/mL, Weight=76.2 kg, Bone Pain=Mild
2021-03-08: Treatment=ADT, PI-RADS=5, PSA=13.9 ng/mL, Weight=76.9 kg, Bone Pain=Mild
2021-06-06: Treatment=ADT, PI-RADS=5, PSA=13.6 ng/mL, Weight=77.7 kg, Bone Pain=Mild


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cd215a5e-8de6-4bdb-b2db-84f5fdb2bde2                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 9b7855e3-9e7e-4316-9cce-6af7b42e3e37
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 9b7855e3-9e7e-4316-9cce-6af7b42e3e37
       Status: Executing Task...
    └── 🤖 Agent: Timeline Planner
            Status: In Progress

# Agent: Timeline Planner
## Task: Use ONLY the structured clinical entries below to construct a timeline of events for patient ID 1.

--- BEGIN STRUCTURED ENTRIES ---
Error fetching or summarizing context: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
--- END STRUCTURED ENTRIES ---

**Rules:**
- Do NOT invent events, symptoms, scans, or dates not included above.
- Each timeline item must match a real dated entry.
- Format each as: - [Date]: [Event Type] — [Details].


...

🚀 Crew: crew
├── 📋 Task: 9b7855e3-9e7e-4316-9cce-6af7b42e3e37
│      Assigned to: Timeline Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Timeline Planner
│           Status: ✅ Completed
├── 📋 Task: ce90562a-ef83-47bd-9875-edaf3da788c4
│      Assigned to: LLM Clinical Summarizer
│      Status: ✅ Completed
│   └── 🤖 Agent: LLM Clinical Summarizer
│           Status: ✅ Completed
│       └── 🔧 Used TrajectoryEvidenceTool (3)
└── 📋 Task: b4c36709-96c3-4232-8a5c-71f841cd78a2
       Status: Executing Task...
    └── 🤖 Agent: Clinical Output Validator
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 9b7855e3-9e7e-4316-9cce-6af7b42e3e37
│      Assigned to: Timeline Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Timeline Planner
│           Status: ✅ Completed
├── 📋 Task: ce90562a-ef83-47bd-9875-edaf3da788c4
│      Assigned to: LLM Clinical Summarizer
│      Status: ✅ Completed
│   └── 🤖 Agent: LLM Clinical Summarizer
│           Status: ✅ Completed
│       └── 🔧 Used TrajectoryEvidenceTool (3)
└── 📋 Task: b4c36709-96c3-4232-8a5c-71f841cd78a2
       Assigned to: Clinical Output Validator
       Status: ✅ Completed
    └── 🤖 Agent: Clinical Output Validator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b4c36709-96c3-4232-8a5c-71f841cd78a2                                                                     │
│  Agent: Clinical Output Validator                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3a444fe9-1197-41de-9ab6-4db618fc4258                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Summary for Patient P1

**1. Clinical Summary for Patient 1** — Patient ID 1, diagnosed with prostate cancer, has been on Androgen Deprivation Therapy (ADT) since March 2020. Over the course of approximately one year, the patient's PSA levels have shown a gradual decrease from 14.7 ng/mL to 13.6 ng/mL. The PI-RADS score has consistently been 5, indicating a high likelihood of clinically significant prostate cancer. Concurrently, the patient's weight has increased from 73.7 kg to 77.7 kg. Bone pain was initially mild, disappeared briefly, but then returned and remained mild.

**2. Clinical Literature Context** — According to [Peppercorn et al., 2025](https://pubmed.ncbi.nlm.nih.gov/40372682/), the management of prostate cancer with ADT can lead to various metabolic changes, which might explain the gradual weight increase observed in the patient. The study emphasizes the importance of monitoring metabolic parameters and adjusting treatment protocols to mitigate potential adverse effects (from *Application of DNA Methylome Analysis to Patients with ME/CFS*).

**3. Patient Timeline**:
- **2020-03-13**: Diagnosed with prostate cancer; Initial PSA=14.7 ng/mL, PI-RADS=5, Weight=73.7 kg; Started ADT; Bone Pain=Mild.
- **2020-06-11**: On ADT; PSA=14.5 ng/mL, PI-RADS=5, Weight=74.5 kg; Bone Pain=None.
- **2020-09-09**: On ADT; PSA=14.4 ng/mL, PI-RADS=5, Weight=75.3 kg; Bone Pain=Mild.
- **2020-12-08**: On ADT; PSA=14.1 ng/mL, PI-RADS=5, Weight=76.2 kg; Bone Pain=Mild.
- **2021-03-08**: On ADT; PSA=13.9 ng/mL, PI-RADS=5, Weight=76.9 kg; Bone Pain=Mild.
- **2021-06-06**: On ADT; PSA=13.6 ng/mL, PI-RADS=5, Weight=77.7 kg; Bone Pain=Mild.

**4. Validation Comments**:
- All information provided in the summary and timeline is directly derived from the structured clinical data provided.
- No hallucinations, missing values, or inconsistencies were noted in the summary or timeline.
- The citation from the TrajectoryEvidenceTool was used to provide context on the potential metabolic effects of ADT, relevant to the patient's weight gain.